# Agri Advisor api

In [1]:
import os
import openai
import sys
sys.path.append('../..')
import conditions

import panel as pn  # GUI
pn.extension()

openai.api_key  =''


In [2]:
#pip install panel


In [3]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

In [4]:
print(conditions.get_products())  # This should output a dictionary


{'Crops': 'This includes grains (such as wheat, corn, rice), vegetables (such as tomatoes, onions, carrots), fruits (such as apples, oranges, bananas), and other crops like cotton, coffee, and sugarcane', 'Livestock': 'This includes animals raised for meat (such as cattle, pigs, chickens), dairy (such as cows, goats, sheep), and other purposes (such as horses for riding, bees for honey production)', 'Poultry': 'This includes chickens, ducks, turkeys, and other domesticated birds raised for meat and eggs', 'Aquaculture': 'This includes fish farming, shrimp farming, and other forms of aquatic animal farming for food production', 'Forestry products': 'This includes timber, wood products, and other products derived from forests and trees', 'Horticulture': 'This includes fruits, vegetables, flowers, ornamental plants, and other crops grown in gardens or greenhouses'}


In [5]:
import json
import openai
from collections import defaultdict

products_file = 'products.json'
categories_file = 'categories.json'

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

def create_categories():
    categories_dict = {
        'Billing': [
            'Unsubscribe or upgrade',
            'Add a payment method',
            'Explanation for charge',
            'Dispute a charge'
        ],
        'Technical Support': [
            'General troubleshooting',
            'Device compatibility',
            'Software updates'
        ],
        'Account Management': [
            'Password reset',
            'Update personal information',
            'Close account',
            'Account security'
        ],
        'General Inquiry': [
            'Product information',
            'Pricing',
            'Feedback',
            'Speak to a human'
        ]
    }

    categories_file = 'categories.json'  
    with open(categories_file, 'w') as file:
        json.dump(categories_dict, file)

    return categories_dict

def get_categories():
    with open(categories_file, 'r') as file:
            categories = json.load(file)
    return categories

def get_products():
    with open(products_file, 'r') as file:
        products = json.load(file)
    return products

def get_products_and_category():
    products_by_category = defaultdict(list)
    products = get_products()
    for category, product_info in products.items():
        if isinstance(product_info, str):
            products_by_category[category].append(product_info)
        else:
            for product_name, product_detail in product_info.items():
                products_by_category[category].append(product_name)
    return dict(products_by_category)

def find_category_and_product_only(user_input, products_and_category):
    delimiter = "####"
    system_message = f"""
    You will be provided with former's queries. 
    The former's query will be delimited with {delimiter} characters.
    Output a Python list of objects, where each object has the following format:
        'agriculture category': <Crops, Livestock, Fisheries, Forestry, Agrochemicals, Agri-inputs, Agri-food processing, Agro-allied products>,
    OR
        'products': <a list of products that must be found in the allowed products list below>

    Where the categories and products must be found in the former's service query.
    If a product is mentioned, it must be associated with the correct category in the allowed products list below.
    If no products or categories are found, output an empty list.

    Allowed products: 

    1. Crops:
       - Cereals (e.g., rice, wheat, corn)
       - Fruits (e.g., apples, oranges, bananas)
       - Vegetables (e.g., tomatoes, potatoes, carrots)
       - Pulses (e.g., lentils, chickpeas, beans)
       - Oilseeds (e.g., soybeans, sunflower, canola)
       - Spices (e.g., pepper, cinnamon, turmeric)

    2. Livestock:
       - Dairy products (e.g., milk, cheese, yogurt)
       - Meat products (e.g., beef, pork, poultry)
       - Eggs
       - Wool and fibers (e.g., wool, silk)
       - Honey

    3. Aquaculture:
       - Fish (e.g., salmon, tilapia, trout)
       - Shrimp
       - Oysters
       - Seaweed

    4. Forestry products:
       - Timber (e.g., pine, oak, teak)
       - Pulp and paper
       - Resins and gums

    5. Horticulture products:
       - Flowers (e.g., roses, lilies, orchids)
       - Nursery plants (e.g., tree saplings, ornamental plants)
       - Medicinal plants
       - Aromatic plants
   
    Only output the list of objects, nothing else.
    """
    messages = [  
        {'role': 'system', 'content': system_message},    
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)

def generate_output_string(category_and_product_list):
    output_string = ''
    for item in category_and_product_list:
        output_string += f"{item}\n"
    return output_string

def read_string_to_list(input_string):
    result = json.loads(input_string)
    return result

user_input = "forestry products types"
response = find_category_and_product_only(user_input, get_products_and_category())
print(response)


[{'agriculture category': 'Forestry'}]


In [6]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [7]:
import panel as pn

# Define function to collect messages
def collect_messages(event):
    # Get user input from the text input widget
    user_input = inp.value
    
    # Process user input and generate response
    response, context = process_user_message(user_input, context)
    
    # Append the user input and response to the panel
    panels.append({'role': 'user', 'content': user_input})
    panels.append({'role': 'assistant', 'content': response})
    
    # Update the conversation panel with the new messages
    interactive_conversation.object = pn.panel(panels)

# Define the initial context
context = [{'role': 'system', 'content': "You are an agriculture advisor."}]

# Create the text input widget
inp = pn.widgets.TextInput(placeholder='Enter text here…')

# Create the conversation button
button_conversation = pn.widgets.Button(name="Send")

# Bind the conversation button to the collect_messages function
button_conversation.on_click(collect_messages)

# Create an empty list to collect display panels
panels = []

# Create a Panel object for the conversation
interactive_conversation = pn.Column(loading_indicator=True, height=300)

# Create the dashboard layout
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    interactive_conversation
)

# Display the dashboard
dashboard


Traceback (most recent call last):
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\panel\viewable.py", line 297, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 115, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\bokeh\document\callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 115, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\bokeh\document\document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_json(self, event_json, references, setter)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\bokeh\document\events.py", line 259, in handle_json
 handler(doc, event_json, references, setter)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\bokeh\document\events.py", line 300, in _handle_json
 cb(event_json["msg_data"])
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\bokeh\document\callbacks.py", line 355, in trigger_json_event
 model._trigger_event(event)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\bokeh\util\callback_manager.py", line 119, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\bokeh\document\callbacks.py", line 247, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\bokeh\document\callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\bokeh\util\callback_manager.py", line 115, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\panel\reactive.py", line 413, in _comm_event
 state._handle_exception(e)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\panel\io\state.py", line 408, in _handle_exception
 raise exception
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\panel\reactive.py", line 411, in _comm_event
 self._process_bokeh_event(doc, event)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\panel\reactive.py", line 348, in _process_bokeh_event
 self._process_event(event)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\panel\widgets\button.py", line 184, in _process_event
 self.clicks += 1
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\param\parameterized.py", line 367, in _f
 instance_param.__set__(obj, val)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\param\parameterized.py", line 369, in _f
 return f(self, obj, val)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\param\__init__.py", line 625, in __set__
 super(Dynamic,self).__set__(obj,val)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\param\parameterized.py", line 369, in _f
 return f(self, obj, val)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\param\parameterized.py", line 1252, in __set__
 obj.param._call_watcher(watcher, event)
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\param\parameterized.py", line 2043, in _call_watcher
 self_._execute_watcher(watcher, (event,))
 File "C:\Users\udhaya\Anaconda3\envs\aim1\lib\site-packages\param\parameterized.py", line 2025, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\udhaya\AppData\Local\Temp\ipykernel_2128\571857810.py", line 9, in collect_messages
 response, context = process_user_message(user_input, context)
NameErr

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Send')
    [2] Column(height=300)

#                 old coding

In [9]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = conditions.find_category_and_product_only(user_input, conditions.get_products_and_category(conditions.get_products()))

    # Step 2: Extract the list of products
    category_and_product_list = conditions.read_string_to_list(category_and_product_response)
    #print(category_and_product_list)
    


    if debug: print("Step 2: Extracted list of products.")

    # Step 3: If products are found, look them up
    product_information = conditions.generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")

    # Step 4: Answer the user question
    system_message = f"""
    . \
    You are a agriculture advisor assistant for a formers. \
    Respond in a friendly and helpful tone, with VERY concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    # Step 5: Put the answer through the Moderation API
    response = openai.Moderation.create(input=final_response)
    moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages

user_input = "mention the types of organic agriculture product"
response,_ = process_user_message(user_input,[])
print(response)



Step 1: Input passed moderation check.


AttributeError: 'str' object has no attribute 'get'